In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -U pypdfium2

In [4]:
import pypdfium2 as pdfium
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

In [5]:
def convert_pdf_to_images(file_path, scale=300/72):

    pdf_file = pdfium.PdfDocument(file_path)
    page_indices = [i for i in range(len(pdf_file))]

    renderer = pdf_file.render(
        pdfium.PdfBitmap.to_pil,
        page_indices = page_indices,
        scale = scale,
    )

    list_final_images = []

    for i, image in zip(page_indices, renderer):

        image_byte_array = BytesIO()
        image.save(image_byte_array, format='jpeg', optimize=True)
        image_byte_array = image_byte_array.getvalue()
        list_final_images.append(dict({i:image_byte_array}))

    return list_final_images

In [6]:
def display_images(list_dict_final_images):

    all_images = [list(data.values())[0] for data in list_dict_final_images]

    for index, image_bytes in enumerate(all_images):

        image = Image.open(BytesIO(image_bytes))
        figure = plt.figure(figsize = (image.width / 100, image.height / 100))

        plt.title(f"----- Page Number {index+1} -----")
        plt.imshow(image)
        plt.axis("off")
        plt.show()

In [7]:
convert_pdf_to_images = convert_pdf_to_images('/content/drive/MyDrive/PDF Text extraction/product_catalogue_for_task2.pdf')

In [9]:
#convert_pdf_to_images
display_images(convert_pdf_to_images)

Output hidden; open in https://colab.research.google.com to view.

**PyPDf2**

In [22]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [23]:
from PyPDF2 import PdfReader

In [24]:
def extract_text_with_pyPDF(PDF_File):

    pdf_reader = PdfReader(PDF_File)

    raw_text = ''

    for i, page in enumerate(pdf_reader.pages):

        text = page.extract_text()
        if text:
            raw_text += text

    return raw_text

In [25]:
text_with_pyPDF = extract_text_with_pyPDF("/content/drive/MyDrive/PDF Text extraction/product_catalogue_for_task2.pdf")
print(text_with_pyPDF)

 
 PRODUKTÜBERSICHT
INDUKTIVE SENSOREN (LVDT) | WIRBELSTROMSENSOREN |  
DIGITALE MESSTASTER, DIGITALE MASSSTÄBE, DIGITALE MAGNETBÄNDER | 
LASERSENSOREN
 
PRODUKTÜBERSICHTeddylab GmbH ist ein hochmodernes und innovatives Unternehmen, das 
sich seit 20 Jahren leidenschaftlich mit der Entwicklung, Konstruktion und Produktion 
von Sensoren zur Erfassung geometrischer Größen bis hin zur kompletten Systemlö -
sung beschäftigt. 
Sitz des Unternehmens ist Otterfing, im Süden von München. Von hier aus erfolgt die 
Entwicklung, die Produktion und der Vertrieb der Sensorik. Zusätzlich können unsere 
Kunden auf ein weltweites Netz an Distributoren zurückgreifen. Langjährige Mitar -
beiter mit großer Applikationserfahrung bieten Ihnen eine kompetente, technische 
Beratung und stellen sicher, die perfekte Lösung für Ihr messtechnisches Problem zu 
finden. 
Das Produktportfolio besteht aus berührend und berührungslos messenden Senso -
ren mit hoher Genauigkeit. Schwerpunkte sind Wirbelstromsensoren s

In [31]:
from PyPDF2 import PdfReader
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# Open the PDF file
with open('/content/drive/MyDrive/PDF Text extraction/product_catalogue_for_task2.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PdfReader(file)

    # Extract text from each page
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()

# Tokenize the text into words
words = word_tokenize(text)

# Tag each word with its part of speech
tagged_words = pos_tag(words)

# Filter out words that are likely to be English
english_words = [word for word, tag in tagged_words if tag.startswith('NN')]

# Join the filtered words back into text
english_text = ' '.join(english_words)

print(english_text)

PRODUKTÜBERSICHT INDUKTIVE SENSOREN LVDT WIRBELSTROMSENSOREN | DIGITALE MESSTASTER DIGITALE MASSSTÄBE DIGITALE MAGNETBÄNDER | LASERSENSOREN PRODUKTÜBERSICHTeddylab GmbH ist ein hochmodernes innovatives Unternehmen das Jahren mit der Entwicklung Konstruktion Produktion von Sensoren zur Erfassung geometrischer Größen bis hin zur Systemlö sung beschäftigt Sitz Unternehmens ist Otterfing Süden von München Von aus erfolgt die Entwicklung Produktion der Vertrieb der Sensorik Zusätzlich Kunden auf ein Netz Distributoren zurückgreifen Langjährige Mitar beiter mit großer Applikationserfahrung Ihnen eine kompetente technische Beratung stellen sicher Lösung für Ihr messtechnisches Problem zu finden Das Produktportfolio besteht aus berührungslos messenden Senso ren mit Genauigkeit Schwerpunkte Wirbelstromsensoren sowie Sensoren LVDT genannt GmbH offizieller Distributor der Magnescale Produkte ehemals Sony Precision T echnology für hochgenaue hochauflösende robuste Sensorik stehen vervollständigen 

Langchain

In [ ]:
!pip install langchain
!pip install nltk
!pip install spacy
!pip install gensim

In [ ]:
! pip install unstructured

In [8]:
from langchain.document_loaders.image import UnstructuredImageLoader

def extract_text_with_langchain_image(list_dict_final_images):

    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []

    for index, image_bytes in enumerate(image_list):

        image = Image.open(BytesIO(image_bytes))
        loader = UnstructuredImageLoader(image)
        data = loader.load()
        raw_text = data[index].page_content

        image_content.append(raw_text)

    return "\n".join(image_content)

In [ ]:
#!pip install pdf2image
#!pip install pdfminer.six
!pip install pillow-heif

In [ ]:
text_with_langchain_image = extract_text_with_langchain_image(convert_pdf_to_images)
print(text_with_langchain_image)

In [18]:
from langchain.document_loaders import UnstructuredFileLoader

def extract_text_with_langchain_pdf(pdf_file):

    loader = UnstructuredFileLoader(pdf_file)
    documents = loader.load()
    pdf_pages_content = '\n'.join(doc.page_content for doc in documents)

    return pdf_pages_content

In [ ]:
text_with_langchain_files = extract_text_with_langchain_pdf("/content/drive/MyDrive/PDF Text extraction/product_catalogue_for_task2.pdf")
print(text_with_langchain_files)